In [1]:
import requests,json
from urllib.parse import quote   #한글인코딩
import folium
import pandas as pd

In [2]:
#파일 읽기
with open('data/roadapikey.txt') as f:
    road_key = f.read()

In [3]:
# 한글은 반드시 인코딩해서 보내야 함
quote('아주대학교')

'%EC%95%84%EC%A3%BC%EB%8C%80%ED%95%99%EA%B5%90'

In [4]:
base_url = 'https://www.juso.go.kr/addrlink/addrLinkApiJsonp.do'
params1=f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2=f'keyword={quote("아주대학교")}&resultType=json'
url =f'{base_url}?{params1}&{params2}'

In [5]:
result =requests.get(url)
result.status_code

200

In [6]:
res = json.loads(result.text[1:-1])

In [7]:
res['results']['juso'][0]

{'detBdNmList': '',
 'engAddr': '199 World cup-ro, Yeongtong-gu, Suwon-si, Gyeonggi-do',
 'rn': '월드컵로',
 'emdNm': '원천동',
 'zipNo': '16502',
 'roadAddrPart2': ' (원천동)',
 'emdNo': '01',
 'sggNm': '수원시 영통구',
 'jibunAddr': '경기도 수원시 영통구 원천동 13-4 아주대학교어학교육원',
 'siNm': '경기도',
 'roadAddrPart1': '경기도 수원시 영통구 월드컵로 199',
 'bdNm': '아주대학교어학교육원',
 'admCd': '4111710200',
 'udrtYn': '0',
 'lnbrMnnm': '13',
 'roadAddr': '경기도 수원시 영통구 월드컵로 199 (원천동)',
 'lnbrSlno': '4',
 'buldMnnm': '199',
 'bdKdcd': '0',
 'liNm': '',
 'rnMgtSn': '411173177012',
 'mtYn': '0',
 'bdMgtSn': '4111710200100130004004722',
 'buldSlno': '0'}

In [8]:
res['results']['juso'][0]['roadAddr']

'경기도 수원시 영통구 월드컵로 199 (원천동)'

In [9]:
gov_list = '아주대학교,동남보건대학,수원 경기대학교,수원 성균관대학교,수원여자대학,용인 경희대학교,수원대학교,수원과학대학'.split(',')
gov_list

['아주대학교',
 '동남보건대학',
 '수원 경기대학교',
 '수원 성균관대학교',
 '수원여자대학',
 '용인 경희대학교',
 '수원대학교',
 '수원과학대학']

In [10]:
road_addr_list =[]
for gov in gov_list:
    params2 = f'keyword={quote(gov)}&resultType=json'
    url =f'{base_url}?{params1}&{params2}'
    result =requests.get(url)
    if result.status_code ==200:
        res = json.loads(result.text[1:-1])
        road_addr_list.append(res['results']['juso'][0]['roadAddr'])  
    else:
        print(result.status_code)

In [11]:
import pandas as pd
df = pd.DataFrame({
    '이름':gov_list,'주소':road_addr_list
})
df

,이름,주소
0,아주대학교,경기도 수원시 영통구 월드컵로 199 (원천동)
1,동남보건대학,경기도 수원시 장안구 천천로74번길 50 (정자동)
2,수원 경기대학교,경기도 수원시 영통구 광교산로 154-42 (이의동)
3,수원 성균관대학교,경기도 수원시 장안구 서부로 2066 (천천동)
4,수원여자대학,경기도 수원시 권선구 온정로 72 (오목천동)
5,용인 경희대학교,경기도 용인시 기흥구 덕영대로 1732 (서천동)
6,수원대학교,경기도 화성시 봉담읍 와우안길 17
7,수원과학대학,경기도 화성시 정남면 세자로 288


### API 주소 가져오기

In [12]:
with open('data/kakaoapikey.txt') as f:
    kakao_key =f.read()
base_url ='https://dapi.kakao.com/v2/local/search/address.json'
addr ='경기도 수원시 영통구 월드컵로 199 (원천동)'                #샘플
url = f'{base_url}?query={quote(addr)}'
header = {'Authorization':f'KakaoAK {kakao_key}'}
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '경기 수원시 영통구 원천동 13-4',
    'b_code': '4111710200',
    'h_code': '4111755000',
    'main_address_no': '13',
    'mountain_yn': 'N',
    'region_1depth_name': '경기',
    'region_2depth_name': '수원시 영통구',
    'region_3depth_h_name': '원천동',
    'region_3depth_name': '원천동',
    'sub_address_no': '4',
    'x': '127.044142224462',
    'y': '37.279204703544'},
   'address_name': '경기 수원시 영통구 월드컵로 199',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '경기 수원시 영통구 월드컵로 199',
    'building_name': '아주대학교어학교육원',
    'main_building_no': '199',
    'region_1depth_name': '경기',
    'region_2depth_name': '수원시 영통구',
    'region_3depth_name': '원천동',
    'road_name': '월드컵로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.044142224462',
    'y': '37.279204703544',
    'zone_no': '16502'},
   'x': '127.044142224462',
   'y': '37.279204703544'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [13]:
result['documents'][0].keys()

dict_keys(['address', 'address_name', 'address_type', 'road_address', 'x', 'y'])

In [14]:
lat = float(result['documents'][0]['y'])
lng = float(result['documents'][0]['x'])
lat,lng

(37.279204703544, 127.044142224462)

In [15]:
df.to_csv('data/수원 대학교',index=False)
df=pd.read_csv('data/수원 대학교')
df

,이름,주소
0,아주대학교,경기도 수원시 영통구 월드컵로 199 (원천동)
1,동남보건대학,경기도 수원시 장안구 천천로74번길 50 (정자동)
2,수원 경기대학교,경기도 수원시 영통구 광교산로 154-42 (이의동)
3,수원 성균관대학교,경기도 수원시 장안구 서부로 2066 (천천동)
4,수원여자대학,경기도 수원시 권선구 온정로 72 (오목천동)
5,용인 경희대학교,경기도 용인시 기흥구 덕영대로 1732 (서천동)
6,수원대학교,경기도 화성시 봉담읍 와우안길 17
7,수원과학대학,경기도 화성시 정남면 세자로 288


In [16]:
lat_list,lng_list =[],[]
for i in df.index:
    url = f'{base_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [17]:
df['위도'] = lat_list
df['경도'] = lng_list
df['color'] = ['red','green','blue','orange','deeppink','yellow','aqua','blueviolet']
df

,이름,주소,위도,경도,color
0,아주대학교,경기도 수원시 영통구 월드컵로 199 (원천동),37.279205,127.044142,red
1,동남보건대학,경기도 수원시 장안구 천천로74번길 50 (정자동),37.296938,126.987000,green
2,수원 경기대학교,경기도 수원시 영통구 광교산로 154-42 (이의동),37.298014,127.033117,blue
3,수원 성균관대학교,경기도 수원시 장안구 서부로 2066 (천천동),37.295976,126.974209,orange
4,수원여자대학,경기도 수원시 권선구 온정로 72 (오목천동),37.252705,126.963307,deeppink
5,용인 경희대학교,경기도 용인시 기흥구 덕영대로 1732 (서천동),37.239800,127.081199,yellow
6,수원대학교,경기도 화성시 봉담읍 와우안길 17,37.210117,126.979480,aqua
7,수원과학대학,경기도 화성시 정남면 세자로 288,37.192033,126.983927,blueviolet


In [18]:
map = folium.Map(location=[df.위도.mean(),df.경도.mean()], zoom_start=12)
for i in df.index:
    folium.Circle(
        location=[df.위도[i],df.경도[i]],
        popup=folium.Popup(df.주소[i],max_width=200),
        tooltip=df.이름[i],
        radius=300,color=df.color[i],fill=True
    ).add_to(map)
title_html ='<h3 align="center" style ="font-size:20px">수원 소재 주요 대학</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map